In [1]:
# | default_exp nets/swin_3d

# Imports

In [2]:
# | export

import numpy as np
import torch
from einops import rearrange, repeat
from huggingface_hub import PyTorchModelHubMixin
from vision_architectures.utils.custom_base_model import CustomBaseModel, model_validator
from torch import nn

from vision_architectures.layers.attention import Attention3DWithMLP
from vision_architectures.layers.embeddings import (
    AbsolutePositionEmbeddings3D,
    PatchEmbeddings3D,
    RelativePositionEmbeddings3D,
)

# Config

In [3]:
# | export


class Swin3DStageConfig(CustomBaseModel):
    depth: int
    window_size: tuple[int, int, int]

    num_heads: int = 4
    mlp_ratio: int = 4
    layer_norm_eps: float = 1e-6
    use_relative_position_bias: bool = False
    patch_merging: dict | None = None
    patch_splitting: dict | None = None

    attn_drop_prob: float = 0.0
    proj_drop_prob: float = 0.0
    mlp_drop_prob: float = 0.0

    _in_dim: int
    _in_patch_size: tuple[int, int, int]
    _attention_dim: int = None
    _out_dim: int
    _out_patch_size: tuple[int, int, int]

    @model_validator(mode="after")
    def validate(self):
        if isinstance(self.patch_merging, dict):
            assert {"merge_window_size", "out_dim_ratio"}.issubset(self.patch_merging), "Missing keys"
        if isinstance(self.patch_splitting, dict):
            assert {"final_window_size", "out_dim_ratio"}.issubset(self.patch_splitting), "Missing keys"
        return self


class Swin3DConfig(CustomBaseModel):
    in_channels: int
    dim: int
    patch_size: tuple[int, int, int]
    stages: list[Swin3DStageConfig]

    image_size: tuple[int, int, int] | None = None  # required for learnable absolute position embeddings
    drop_prob: float = 0.0
    embed_spacing_info: bool = False
    use_absolute_position_embeddings: bool = True
    learnable_absolute_position_embeddings: bool = False

    def populate(self):
        dim = self.dim
        patch_size = self.patch_size

        # Prepare config based on provided values
        for i in range(len(self.stages)):
            stage = self.stages[i]
            stage._in_dim = dim
            stage._in_patch_size = patch_size
            if stage.patch_merging is not None:
                dim *= stage.patch_merging["out_dim_ratio"]
                stage._attention_dim = dim  # attention will happen after merging
                patch_size = tuple(
                    [patch * window for patch, window in zip(patch_size, stage.patch_merging["merge_window_size"])]
                )
            if stage.patch_splitting is not None:
                stage._attention_dim = dim  # attention will happen before splitting
                dim //= stage.patch_splitting["out_dim_ratio"]
                patch_size = tuple(
                    [patch * window for patch, window in zip(patch_size, stage.patch_splitting["final_window_size"])]
                )
            if stage._attention_dim is None:
                stage._attention_dim = dim  # In case it is not yet set
            stage._out_dim = dim
            stage._out_patch_size = patch_size

    @model_validator(mode="after")
    def validate(self):
        self.populate()

        # test divisibility of dim with number of attention heads
        for stage in self.stages:
            assert (
                stage._out_dim % stage.num_heads == 0
            ), f"stage._out_dim {stage._out_dim} is not divisible by stage.num_heads {stage.num_heads}"

        # test population of image_size field iff the absolute position embeddings are relative
        if self.learnable_absolute_position_embeddings:
            assert (
                self.image_size is not None
            ), "Please provide image_size if absolute position embeddings are learnable"

        return self


class Swin3DMIMConfig(Swin3DConfig):
    mim: dict

In [4]:
test_config = Swin3DConfig.model_validate(
    {
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "embed_spacing_info": False,
        "image_size": (32, 512, 512),
        "dim": 36,
        "stages": [
            {
                "patch_merging": None,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
        ],
    }
)

test_config


Swin3DConfig(
    in_channels=1,
    dim=36,
    patch_size=(1, 8, 8),
    stages=[
        Swin3DStageConfig(
            depth=1,
            window_size=(4, 4, 4),
            num_heads=4,
            mlp_ratio=4,
            layer_norm_eps=1e-06,
            use_relative_position_bias=False,
            patch_merging=None,
            patch_splitting=None,
            attn_drop_prob=0.0,
            proj_drop_prob=0.0,
            mlp_drop_prob=0.0
        ),
        Swin3DStageConfig(
            depth=3,
            window_size=(4, 4, 4),
            num_heads=4,
            mlp_ratio=4,
            layer_norm_eps=1e-06,
            use_relative_position_bias=True,
            patch_merging={'merge_window_size': (2, 2, 2), 'out_dim_ratio': 3},
            patch_splitting=None,
            attn_drop_prob=0.0,
            proj_drop_prob=0.0,
            mlp_drop_prob=0.0
        ),
        Swin3DStageConfig(
            depth=1,
            window_size=(4, 4, 4),
            num_h

# Architecture

### Basic Layers

In [5]:
# | export


class Swin3DLayer(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio, layer_norm_eps, window_size, use_relative_position_bias):
        super().__init__()

        qkv_relative_position_bias = None
        if use_relative_position_bias:
            qkv_relative_position_bias = RelativePositionEmbeddings3D(num_heads, window_size)

        self.attn = Attention3DWithMLP(
            dim=dim,
            num_q_heads=num_heads,
            mlp_ratio=mlp_ratio,
            qkv_relative_position_bias=qkv_relative_position_bias,
            activation="gelu",
            norm_location="pre",
            layer_norm_eps=layer_norm_eps,
        )

        self.window_size = window_size

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)
        _, num_patches_z, num_patches_y, num_patches_x, _ = hidden_states.shape

        # Perform windowing
        window_size_z, window_size_y, window_size_x = self.window_size
        num_windows_z, num_windows_y, num_windows_x = (
            num_patches_z // window_size_z,
            num_patches_y // window_size_y,
            num_patches_x // window_size_x,
        )
        hidden_states = rearrange(
            hidden_states,
            "b (num_windows_z window_size_z) (num_windows_y window_size_y) (num_windows_x window_size_x) dim -> "
            "(b num_windows_z num_windows_y num_windows_x) window_size_z window_size_y window_size_x dim ",
            num_windows_z=num_windows_z,
            num_windows_y=num_windows_y,
            num_windows_x=num_windows_x,
            window_size_z=window_size_z,
            window_size_y=window_size_y,
            window_size_x=window_size_x,
        )

        hidden_states = self.attn(hidden_states, hidden_states, hidden_states, channels_first=False)

        # Undo windowing
        output = rearrange(
            hidden_states,
            "(b num_windows_z num_windows_y num_windows_x) window_size_z window_size_y window_size_x dim -> "
            "b (num_windows_z window_size_z) (num_windows_y window_size_y) (num_windows_x window_size_x) dim",
            num_windows_z=num_windows_z,
            num_windows_y=num_windows_y,
            num_windows_x=num_windows_x,
            window_size_z=window_size_z,
            window_size_y=window_size_y,
            window_size_x=window_size_x,
        )

        return output

In [6]:
test = Swin3DLayer(64, 4, 256, 1e-6, (4, 4, 4), True)
display(test)
display(test(torch.randn(2, 4, 4, 4, 64)).shape)


Swin3DLayer(
  (attn): Attention3DWithMLP(
    (attn): Attention3D(
      (W_q): Linear(in_features=64, out_features=64, bias=True)
      (W_k): Linear(in_features=64, out_features=64, bias=True)
      (W_v): Linear(in_features=64, out_features=64, bias=True)
      (proj): Linear(in_features=64, out_features=64, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
      (relative_position_bias): RelativePositionEmbeddings3D()
    )
    (layernorm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
    (mlp): Attention3DMLP(
      (dense1): Linear(in_features=64, out_features=16384, bias=True)
      (act): GELU(approximate='none')
      (dense2): Linear(in_features=16384, out_features=64, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (layernorm2): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
  )
)

torch.Size([2, 4, 4, 4, 64])

### Stage layers

In [7]:
# | export


class Swin3DBlock(nn.Module):
    def __init__(self, stage_config: Swin3DStageConfig):
        super().__init__()

        self.stage_config = stage_config
        self.w_layer = Swin3DLayer(
            stage_config._out_dim,
            stage_config.num_heads,
            stage_config.mlp_ratio,
            stage_config.layer_norm_eps,
            stage_config.window_size,
            stage_config.use_relative_position_bias,
        )
        self.sw_layer = Swin3DLayer(
            stage_config._out_dim,
            stage_config.num_heads,
            stage_config.mlp_ratio,
            stage_config.layer_norm_eps,
            stage_config.window_size,
            stage_config.use_relative_position_bias,
        )

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)

        layer_outputs = []

        # First layer
        hidden_states = self.w_layer(hidden_states)
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)

        layer_outputs.append(hidden_states)

        # Shift windows
        window_size_z, window_size_y, window_size_x = self.stage_config.window_size
        shifts = (window_size_z // 2, window_size_y // 2, window_size_x // 2)
        hidden_states = torch.roll(hidden_states, shifts=shifts, dims=(1, 2, 3))
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)

        # Second layer
        hidden_states = self.sw_layer(hidden_states)
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)

        # Reverse window shift
        shifts = tuple(-shift for shift in shifts)
        hidden_states = torch.roll(hidden_states, shifts=shifts, dims=(1, 2, 3))
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)

        layer_outputs.append(hidden_states)

        return hidden_states, layer_outputs

In [8]:
test_stage_config = Swin3DStageConfig.model_validate(
    {
        "depth": 4,
        "num_heads": 4,
        "mlp_ratio": 4,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
    }
)
test_stage_config._out_dim = 64

test = Swin3DBlock(test_stage_config)
display(test)
o = test(torch.randn(2, 4, 4, 4, 64))
display((o[0].shape, (o[1][0].shape, o[1][1].shape)))


Swin3DBlock(
  (w_layer): Swin3DLayer(
    (attn): Attention3DWithMLP(
      (attn): Attention3D(
        (W_q): Linear(in_features=64, out_features=64, bias=True)
        (W_k): Linear(in_features=64, out_features=64, bias=True)
        (W_v): Linear(in_features=64, out_features=64, bias=True)
        (proj): Linear(in_features=64, out_features=64, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
        (relative_position_bias): RelativePositionEmbeddings3D()
      )
      (layernorm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention3DMLP(
        (dense1): Linear(in_features=64, out_features=256, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=256, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
    )
  )
  (sw_layer): Swin3DLayer(
    (attn): Attention3DWithMLP(
      (attn): Attention3D(
    

(torch.Size([2, 4, 4, 4, 64]), (torch.Size([2, 4, 4, 4, 64]), torch.Size([2, 4, 4, 4, 64])))

In [9]:
# | export


class Swin3DPatchMerging(nn.Module):
    def __init__(self, merge_window_size, in_dim, out_dim):
        super().__init__()

        self.merge_window_size = merge_window_size

        in_dim = in_dim * np.prod(merge_window_size)
        self.layer_norm = nn.LayerNorm(in_dim)
        self.proj = nn.Linear(in_dim, out_dim)

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)

        window_size_z, window_size_y, window_size_x = self.merge_window_size

        hidden_states = rearrange(
            hidden_states,
            "b (new_num_patches_z window_size_z) (new_num_patches_y window_size_y) (new_num_patches_x window_size_x) dim -> "
            "b new_num_patches_z new_num_patches_y new_num_patches_x (window_size_z window_size_y window_size_x dim)",
            window_size_z=window_size_z,
            window_size_y=window_size_y,
            window_size_x=window_size_x,
        )

        hidden_states = self.layer_norm(hidden_states)
        hidden_states = self.proj(hidden_states)
        return hidden_states

In [10]:
test_stage_config = Swin3DStageConfig.model_validate(
    {
        "patch_merging": {
            "merge_window_size": (2, 2, 2),
            "out_dim_ratio": 3,
        },
        "depth": 4,
        "num_heads": 4,
        "intermediate_size": 256,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
    }
)
test_stage_config._in_dim = 64
test_stage_config._out_dim = 64 * 3

test = Swin3DPatchMerging(
    test_stage_config.patch_merging["merge_window_size"],
    test_stage_config._in_dim,
    test_stage_config._out_dim,
)
display(test)
display(test(torch.randn(2, 4, 4, 4, 64)).shape)


Swin3DPatchMerging(
  (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (proj): Linear(in_features=512, out_features=192, bias=True)
)

torch.Size([2, 2, 2, 2, 192])

In [11]:
# | export


class Swin3DStage(nn.Module):
    def __init__(self, stage_config: Swin3DStageConfig):
        super().__init__()

        self.config = stage_config

        self.patch_merging = None
        if stage_config.patch_merging is not None:
            self.patch_merging = Swin3DPatchMerging(
                stage_config.patch_merging["merge_window_size"],
                stage_config._in_dim,
                stage_config._out_dim,
            )

        self.blocks = nn.ModuleList(
            [Swin3DBlock(stage_config) for _ in range(stage_config.depth)],
        )

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)

        if self.patch_merging:
            hidden_states = self.patch_merging(hidden_states)
            # (b, new_num_patches_z, new_num_patches_y, new_num_patches_x, new_dim)

        layer_outputs = []
        for layer_module in self.blocks:
            hidden_states, _layer_outputs = layer_module(hidden_states)
            # (b, new_num_patches_z, new_num_patches_y, new_num_patches_x, new_dim)
            layer_outputs.extend(_layer_outputs)

        return hidden_states, layer_outputs

In [12]:
test_stage_config = Swin3DStageConfig.model_validate(
    {
        "patch_merging": {
            "merge_window_size": (2, 2, 2),
            "out_dim_ratio": 3,
        },
        "depth": 2,
        "num_heads": 4,
        "mlp_ratio": 4,
        "layer_norm_eps": 1e-6,
        "window_size": (4, 4, 4),
        "use_relative_position_bias": True,
    }
)
test_stage_config._in_dim = 48
test_stage_config._out_dim = 48 * 3

test = Swin3DStage(test_stage_config)
display(test)
o = test(torch.randn(2, 8, 8, 8, 48))
display((o[0].shape, [x.shape for x in o[1]]))


Swin3DStage(
  (patch_merging): Swin3DPatchMerging(
    (layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (proj): Linear(in_features=384, out_features=144, bias=True)
  )
  (blocks): ModuleList(
    (0-1): 2 x Swin3DBlock(
      (w_layer): Swin3DLayer(
        (attn): Attention3DWithMLP(
          (attn): Attention3D(
            (W_q): Linear(in_features=144, out_features=144, bias=True)
            (W_k): Linear(in_features=144, out_features=144, bias=True)
            (W_v): Linear(in_features=144, out_features=144, bias=True)
            (proj): Linear(in_features=144, out_features=144, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (relative_position_bias): RelativePositionEmbeddings3D()
          )
          (layernorm1): LayerNorm((144,), eps=1e-06, elementwise_affine=True)
          (mlp): Attention3DMLP(
            (dense1): Linear(in_features=144, out_features=576, bias=True)
            (act): GELU(approximate='none')
  


(
    torch.Size([2, 4, 4, 4, 144]),
    [
        torch.Size([2, 4, 4, 4, 144]),
        torch.Size([2, 4, 4, 4, 144]),
        torch.Size([2, 4, 4, 4, 144]),
        torch.Size([2, 4, 4, 4, 144])
    ]
)

### Encoder

In [13]:
# | export


class Swin3DEncoder(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: Swin3DConfig):
        super().__init__()

        self.stages = nn.ModuleList([Swin3DStage(stage_config) for stage_config in config.stages])

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (b, num_patches_z, num_patches_y, num_patches_x, dim)

        stage_outputs, layer_outputs = [], []
        for stage_module in self.stages:
            hidden_states, _layer_outputs = stage_module(hidden_states)
            # (b, new_num_patches_z, new_num_patches_y, new_num_patches_x, dim)

            stage_outputs.append(hidden_states)
            layer_outputs.extend(_layer_outputs)

        return hidden_states, stage_outputs, layer_outputs

In [14]:
test_config = Swin3DConfig.model_validate(
    {
        "dim": 32,
        "in_channels": 32,
        "patch_size": (1, 1, 1),
        "stages": [
            {
                "patch_merging": None,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
        ],
    }
)

test = Swin3DEncoder(test_config)
display(test)
o = test(torch.randn(2, 16, 16, 16, 32))
display((o[0].shape, [x.shape for x in o[1]], [x.shape for x in o[2]]))


Swin3DEncoder(
  (stages): ModuleList(
    (0): Swin3DStage(
      (blocks): ModuleList(
        (0): Swin3DBlock(
          (w_layer): Swin3DLayer(
            (attn): Attention3DWithMLP(
              (attn): Attention3D(
                (W_q): Linear(in_features=32, out_features=32, bias=True)
                (W_k): Linear(in_features=32, out_features=32, bias=True)
                (W_v): Linear(in_features=32, out_features=32, bias=True)
                (proj): Linear(in_features=32, out_features=32, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
              )
              (layernorm1): LayerNorm((32,), eps=1e-06, elementwise_affine=True)
              (mlp): Attention3DMLP(
                (dense1): Linear(in_features=32, out_features=128, bias=True)
                (act): GELU(approximate='none')
                (dense2): Linear(in_features=128, out_features=32, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
        


(
    torch.Size([2, 8, 8, 8, 96]),
    [torch.Size([2, 16, 16, 16, 32]), torch.Size([2, 8, 8, 8, 96])],
    [
        torch.Size([2, 16, 16, 16, 32]),
        torch.Size([2, 16, 16, 16, 32]),
        torch.Size([2, 8, 8, 8, 96]),
        torch.Size([2, 8, 8, 8, 96]),
        torch.Size([2, 8, 8, 8, 96]),
        torch.Size([2, 8, 8, 8, 96]),
        torch.Size([2, 8, 8, 8, 96]),
        torch.Size([2, 8, 8, 8, 96])
    ]
)

# Models

In [15]:
# | export


class Swin3DModel(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: Swin3DConfig):
        super().__init__()

        self.config = config

        self.patchify = PatchEmbeddings3D(config.patch_size, config.in_channels, config.dim)
        self.absolute_position_embeddings = AbsolutePositionEmbeddings3D(config.dim, learnable=False)
        self.encoder = Swin3DEncoder(config)

    def _get_grid_size(self, image_size):
        grid_size = (
            image_size[0] // self.config.patch_size[0],
            image_size[1] // self.config.patch_size[1],
            image_size[2] // self.config.patch_size[2],
        )
        return grid_size

    def forward(
        self,
        pixel_values: torch.Tensor,
        spacings: torch.Tensor,
        mask_patches: torch.Tensor = None,
        mask_token: torch.Tensor = None,
    ):
        # pixel_values: (b, c, z, y, x)
        # spacings: (b, 3)
        # mask_patches: (num_patches_z, num_patches_y, num_patches_x)

        embeddings = self.patchify(pixel_values)
        # (b, dim, num_patches_z, num_patches_y, num_patches_x)

        if mask_patches is not None:
            # mask_patches (binary mask): (b, num_patches_z, num_patches_y, num_patches_x)
            # mask_token: (1, dim, 1, 1, 1)
            mask_patches = repeat(mask_patches, "b z y x -> b d z y x", d=embeddings.shape[1])
            embeddings = (embeddings * (1 - mask_patches)) + (mask_patches * mask_token)

        absolute_position_embeddings = self.absolute_position_embeddings(
            batch_size=embeddings.shape[0], grid_size=embeddings.shape[2:], spacings=spacings
        )
        # (b, dim, num_patches_z, num_patches_y, num_patches_x)
        embeddings = embeddings + absolute_position_embeddings
        # (b, dim, num_patches_z, num_patches_y, num_patches_x)

        embeddings = rearrange(embeddings, "b e nz ny nx -> b nz ny nx e")
        # (b, num_patches_z, num_patches_y, num_patches_x, dim)

        encoded, stage_outputs, layer_outputs = self.encoder(embeddings)
        # encoded: (b, new_num_patches_z, new_num_patches_y, new_num_patches_x, dim)
        # stage_outputs, layer_outputs: list of (b, some_num_patches_z, some_num_patches_y, some_num_patches_x, dim)

        encoded = rearrange(encoded, "b nz ny nx d -> b d nz ny nx")
        # (b, dim, new_num_patches_z, new_num_patches_y, new_num_patches_x)

        for i in range(len(stage_outputs)):
            stage_outputs[i] = rearrange(stage_outputs[i], "b nz ny nx d -> b d nz ny nx")
            # (b, dim, some_num_patches_z, some_num_patches_y, some_num_patches_x)

        for i in range(len(layer_outputs)):
            layer_outputs[i] = rearrange(layer_outputs[i], "b nz ny nx d -> b d nz ny nx")
            # (b, dim, some_num_patches_z, some_num_patches_y, some_num_patches_x)

        return encoded, stage_outputs, layer_outputs

In [16]:
test_config = Swin3DConfig.model_validate(
    {
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "embed_spacing_info": False,
        "image_size": (32, 512, 512),
        "dim": 36,
        "stages": [
            {
                "patch_merging": None,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
        ],
    }
)

test = Swin3DModel(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 512, 512),
    torch.randn(2, 3),
)
display((o[0].shape, [x.shape for x in o[1]], [x.shape for x in o[2]]))


Swin3DModel(
  (patchify): PatchEmbeddings3D(
    (patch_embeddings): Conv3d(1, 36, kernel_size=(1, 8, 8), stride=(1, 8, 8))
    (normalization): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
  )
  (absolute_position_embeddings): AbsolutePositionEmbeddings3D()
  (encoder): Swin3DEncoder(
    (stages): ModuleList(
      (0): Swin3DStage(
        (blocks): ModuleList(
          (0): Swin3DBlock(
            (w_layer): Swin3DLayer(
              (attn): Attention3DWithMLP(
                (attn): Attention3D(
                  (W_q): Linear(in_features=36, out_features=36, bias=True)
                  (W_k): Linear(in_features=36, out_features=36, bias=True)
                  (W_v): Linear(in_features=36, out_features=36, bias=True)
                  (proj): Linear(in_features=36, out_features=36, bias=True)
                  (proj_drop): Dropout(p=0.0, inplace=False)
                )
                (layernorm1): LayerNorm((36,), eps=1e-06, elementwise_affine=True)
             


(
    torch.Size([2, 324, 8, 16, 16]),
    [
        torch.Size([2, 36, 32, 64, 64]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 324, 8, 16, 16])
    ],
    [
        torch.Size([2, 36, 32, 64, 64]),
        torch.Size([2, 36, 32, 64, 64]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 108, 16, 32, 32]),
        torch.Size([2, 324, 8, 16, 16]),
        torch.Size([2, 324, 8, 16, 16])
    ]
)

# For MIM

In [17]:
# | export


class Swin3DMIMDecoder(nn.Module):
    def __init__(self, config: Swin3DMIMConfig):
        super().__init__()

        self.image_size = config.image_size
        self.in_channels = config.in_channels

        dim = config.stages[-1]._out_dim
        patch_size = config.stages[-1]._out_patch_size

        out_dim = np.prod(patch_size) * self.in_channels
        self.final_patch_size = patch_size

        self.decoder = nn.Conv3d(dim, out_dim, kernel_size=1)

    def forward(self, encodings: torch.Tensor):
        # encodings: (b, dim, num_patches_z, num_patches_y, num_patches_x)

        decoded = self.decoder(encodings)
        # (b, new_dim, num_patches_z, num_patches_y, num_patches_x)

        decoded = rearrange(
            decoded,
            "b (c pz py px) nz ny nx -> b c (nz pz) (ny py) (nx px)",
            c=self.in_channels,
            pz=self.final_patch_size[0],
            py=self.final_patch_size[1],
            px=self.final_patch_size[2],
        )
        # (b, c, z, y, x)

        return decoded

In [18]:
test_config = Swin3DMIMConfig.model_validate(
    {
        "dim": 36,
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "embed_spacing_info": False,
        "image_size": (32, 512, 512),
        "stages": [
            {
                "patch_merging": None,
                "depth": 2,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
                "depth": 6,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
        ],
        "mim": {},
    }
)

test = Swin3DMIMDecoder(test_config)
display(test)
display(test(torch.randn(2, 108, 16, 32, 32)).shape)


Swin3DMIMDecoder(
  (decoder): Conv3d(108, 512, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

torch.Size([2, 1, 32, 512, 512])

In [19]:
# | export


class Swin3DMIM(nn.Module, PyTorchModelHubMixin):
    def __init__(self, config: Swin3DMIMConfig):
        super().__init__()

        self.config = config

        self.swin = Swin3DModel(config)
        self.decoder = Swin3DMIMDecoder(config)

        self.mask_token = nn.Parameter(torch.randn(1, config.dim, 1, 1, 1))

    def forward(self, pixel_values: torch.Tensor, spacings: torch.Tensor):
        b = pixel_values.shape[0]

        mask_ratio = self.config.mim["mask_ratio"]
        mask_grid_size = self.config.mim["mask_grid_size"]
        num_patches = np.prod(mask_grid_size)
        mask_patches = []
        for _ in range(b):
            _mask_patches = torch.zeros(num_patches, dtype=torch.int8, device=pixel_values.device)
            _mask_patches[: int(mask_ratio * num_patches)] = 1
            _mask_patches = _mask_patches[torch.randperm(num_patches)]
            _mask_patches = rearrange(
                _mask_patches, "(z y x) -> z y x", z=mask_grid_size[0], y=mask_grid_size[1], x=mask_grid_size[2]
            )
            mask_patches.append(_mask_patches)
        mask_patches: torch.Tensor = torch.stack(mask_patches, dim=0)

        grid_size = tuple([size // patch for size, patch in zip(self.config.image_size, self.config.patch_size)])
        assert all(
            [x % y == 0 for x, y in zip(grid_size, mask_grid_size)]
        ), "Mask grid size must divide image grid size"
        mask_patches = repeat(
            mask_patches,
            "b z y x -> b (z gz) (y gy) (x gx)",
            gz=grid_size[0] // mask_grid_size[0],
            gy=grid_size[1] // mask_grid_size[1],
            gx=grid_size[2] // mask_grid_size[2],
        )

        encodings, _, _ = self.swin(pixel_values, spacings, mask_patches, self.mask_token)

        decoded = self.decoder(encodings)

        loss = nn.functional.l1_loss(decoded, pixel_values, reduction="none")
        mask = repeat(
            mask_patches,
            "b z y x -> b (z pz) (y py) (x px)",
            pz=self.config.patch_size[0],
            py=self.config.patch_size[1],
            px=self.config.patch_size[2],
        )
        loss = (loss * mask).sum() / ((mask.sum() + 1e-5) * self.config.in_channels)

        return decoded, loss, mask

In [20]:
test_config = Swin3DMIMConfig.model_validate(
    {
        "dim": 36,
        "patch_size": (1, 8, 8),
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "embed_spacing_info": False,
        "image_size": (32, 512, 512),
        "stages": [
            {
                "patch_merging": None,
                "depth": 2,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": False,
            },
            {
                "depth": 6,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 3,
                },
            },
        ],
        "mim": {
            "mask_ratio": 0.8,
            "mask_grid_size": (8, 8, 8),
        },
    }
)

test = Swin3DMIM(test_config)
display(test)
o = test(
    torch.randn(2, 1, 32, 512, 512),
    torch.randn(2, 3),
)
display((o[0].shape, o[1], o[2].shape))


Swin3DMIM(
  (swin): Swin3DModel(
    (patchify): PatchEmbeddings3D(
      (patch_embeddings): Conv3d(1, 36, kernel_size=(1, 8, 8), stride=(1, 8, 8))
      (normalization): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
    )
    (absolute_position_embeddings): AbsolutePositionEmbeddings3D()
    (encoder): Swin3DEncoder(
      (stages): ModuleList(
        (0): Swin3DStage(
          (blocks): ModuleList(
            (0-1): 2 x Swin3DBlock(
              (w_layer): Swin3DLayer(
                (attn): Attention3DWithMLP(
                  (attn): Attention3D(
                    (W_q): Linear(in_features=36, out_features=36, bias=True)
                    (W_k): Linear(in_features=36, out_features=36, bias=True)
                    (W_v): Linear(in_features=36, out_features=36, bias=True)
                    (proj): Linear(in_features=36, out_features=36, bias=True)
                    (proj_drop): Dropout(p=0.0, inplace=False)
                  )
                  (layernorm1):


(
    torch.Size([2, 1, 32, 512, 512]),
    tensor(2.0491, grad_fn=<DivBackward0>),
    torch.Size([2, 32, 512, 512])
)

# Some more tests

### Overfitting tests

In [21]:
from tqdm.auto import tqdm

sample_spacings = torch.tensor([[1, 0.1, 0.1], [2, 0.2, 0.2], [3, 0.3, 0.3]])
sample_batch = torch.rand(3, 1, 16, 128, 128)
sample_config = Swin3DMIMConfig.model_validate(
    {
        "patch_size": (1, 4, 4),
        "dim": 12,
        "in_channels": 1,
        "use_absolute_position_embeddings": True,
        "learnable_absolute_position_embeddings": False,
        "embed_spacing_info": False,
        "image_size": (16, 128, 128),
        "stages": [
            {
                "patch_merging": None,
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 4,
                },
                "depth": 3,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
            {
                "patch_merging": {
                    "merge_window_size": (2, 2, 2),
                    "out_dim_ratio": 4,
                },
                "depth": 1,
                "num_heads": 4,
                "mlp_ratio": 4,
                "layer_norm_eps": 1e-6,
                "window_size": (4, 4, 4),
                "use_relative_position_bias": True,
            },
        ],
        "mim": {
            "mask_ratio": 0.7,
            "mask_grid_size": (8, 8, 8),
        },
    }
)

model = Swin3DMIM(sample_config)

sum(x.numel() for x in model.swin.parameters()), sum(x.numel() for x in model.decoder.parameters())

(1156622, 197632)

In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [23]:
# sample_batch = sample_batch.cuda()
# sample_spacings = sample_spacings.cuda()
# model = model.cuda()

In [24]:
for i in tqdm(range(200)):
    optimizer.zero_grad()
    output = model(sample_batch, sample_spacings)
    print(f"Loss: {output[1]:f}\tLR: {scheduler.get_last_lr()[0]:f}")
    output[1].backward()
    optimizer.step()
    scheduler.step()

  0%|          | 0/200 [00:00<?, ?it/s]

Loss: 1.757393	LR: 0.500000
Loss: 1.993328	LR: 0.500000
Loss: 2.322113	LR: 0.500000
Loss: 2.046316	LR: 0.500000
Loss: 2.380407	LR: 0.500000
Loss: 2.319118	LR: 0.450000
Loss: 2.054285	LR: 0.450000
Loss: 2.033215	LR: 0.450000
Loss: 1.867610	LR: 0.450000
Loss: 1.678152	LR: 0.450000
Loss: 1.566827	LR: 0.405000
Loss: 1.296888	LR: 0.405000
Loss: 1.415558	LR: 0.405000
Loss: 1.250074	LR: 0.405000
Loss: 1.269548	LR: 0.405000
Loss: 1.309326	LR: 0.364500
Loss: 1.451795	LR: 0.364500
Loss: 1.375090	LR: 0.364500
Loss: 1.024246	LR: 0.364500
Loss: 1.032505	LR: 0.364500
Loss: 1.033040	LR: 0.328050
Loss: 0.922670	LR: 0.328050
Loss: 0.923329	LR: 0.328050
Loss: 0.938134	LR: 0.328050
Loss: 0.902660	LR: 0.328050
Loss: 0.894042	LR: 0.295245
Loss: 0.856852	LR: 0.295245
Loss: 0.850145	LR: 0.295245
Loss: 0.863580	LR: 0.295245
Loss: 0.881036	LR: 0.295245
Loss: 0.851702	LR: 0.265721
Loss: 0.804387	LR: 0.265721
Loss: 0.786686	LR: 0.265721
Loss: 0.787701	LR: 0.265721
Loss: 0.791564	LR: 0.265721
Loss: 0.798318	LR: 0

# nbdev

In [25]:
!nbdev_export